### Import necessary libraries

In [ ]:
import mdtraj as md
import MDAnalysis as mda
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
from sklearn.cluster import KMeans
%matplotlib inline

# Features Extraction 

_This section outlines the process of extracting relevant features from the molecular dynamics simulation data of the protein. Features like dihedral angles, number of hydrogen bonds, RMSD, Rg, and SASA are crucial descriptors of the protein's conformational states. By extracting these features, we can gain insights into the structural and dynamic properties of the protein over time._

### Diherdral Angles

_Dihedral angles describe the rotations between atoms in a molecule, particularly focusing on backbone angles like phi (ϕ) and psi (ψ)_

In [ ]:
# Load the trajectory
traj = md.load('md_0_1_noPBC.xtc', top='topology.pdb')

# Compute phi and psi dihedral angles
_, phi_angles = md.compute_phi(traj)
_, psi_angles = md.compute_psi(traj)

# Prepare the DataFrame for the dihedral angles
phi_df = pd.DataFrame(phi_angles)
psi_df = pd.DataFrame(psi_angles)
angles_df = pd.concat([phi_df, psi_df], axis=1)

#Remove the column labels
angles_df.columns = range(angles_df.shape[1])

# Save the angles DataFrame to a CSV file
angles_df.to_csv('dihedral_angles.csv', index=False)

# Plot the angles
plt.figure(figsize=(10, 6))
for i in range(phi_angles.shape[1]):
    plt.plot(phi_angles[:, i], label=f'Phi {i + 1}')
for i in range(psi_angles.shape[1]):
    plt.plot(psi_angles[:, i], label=f'Psi {i + 1}')
plt.xlabel('Frame')
plt.ylabel('Dihedral Angle (radians)')
plt.legend()
plt.title('Phi and Psi Dihedral Angles')
plt.savefig('dihedral_angles_plot.png')
plt.show()


In [ ]:
angles_df.head()

### Hydrogen Bond

_Tracking hydrogen bonds over time can reveal insights into the protein's folding and stability._

In [ ]:
# Identify hydrogen bonds using the wernet_nilsson method
hbonds = md.wernet_nilsson(traj)

# Count the number of hydrogen bonds in each frame
hbond_counts = [len(frame_hbonds) for frame_hbonds in hbonds]

# Create DataFrame for hydrogen bonds
hbond_df = pd.DataFrame(hbond_counts)

# Save hydrogen bond counts to a CSV file
hbond_df.to_csv('hbond_counts.csv', index=False)

# Plot the number of hydrogen bonds in each frame
plt.figure(figsize=(10, 6))
plt.plot(hbond_df.index, hbond_df[0], label='Hydrogen Bonds', color='blue')
plt.xlabel('Frame')
plt.ylabel('Number of Hydrogen Bonds')
plt.title('Number of Hydrogen Bonds in Each Frame')
plt.legend()
plt.grid(True)
plt.savefig('hbond_plot.png')
plt.show()

In [ ]:
hbond_df.head()

### Radius of Gyration 

_Radius of gyration (Rg) measures the compactness of the protein structure by calculating the root mean square distance of atoms from their center of mass. In this section, Rg will be computed to analyze how the compactness of the protein changes during the simulation. Fluctuations in Rg can indicate folding and unfolding events or transitions between different conformational states._

In [ ]:
# Compute the radius of gyration for each frame
radii_of_gyration = md.compute_rg(traj)

# Create DataFrame for radius of gyration
rg_df = pd.DataFrame(radii_of_gyration)

# Save the radius of gyration values to a CSV file
rg_df.to_csv('Radius_Of_Gyration.csv', index=False)

# Plot the radius of gyration over time
plt.figure(figsize=(10, 6))
plt.plot(rg_df.index, rg_df[0], label='Radius of Gyration', color='blue')
plt.xlabel('Frame')
plt.ylabel('Radius of Gyration (nm)')
plt.title('Radius of Gyration Over Time')
plt.legend()
plt.grid(True)
plt.savefig('radius_of_gyration_plot.png')
plt.show()

In [ ]:
rg_df.head()

### RMSD

_RMSD measures the average deviation of atomic positions in a protein over time relative to a reference structure. This section calculates the RMSD of the protein's backbone atoms to evaluate its structural stability during the simulation. A high RMSD might indicate significant conformational changes, while a low RMSD suggests a stable structure._

In [ ]:
# Calculate RMSD for all frames relative to the first frame
rmsds = md.rmsd(traj, traj, 0)

# Create DataFrame for RMSD
rmsd_df = pd.DataFrame(rmsds)

# Save RMSD values to a CSV file
rmsd_df.to_csv('Rmsd_values.csv', index=False)

# Plot RMSD|
plt.figure(figsize=(10, 6))
plt.plot(rmsd_df.index, rmsd_df[0], label='RMSD')
plt.xlabel('Frame')
plt.ylabel('RMSD (nm)')
plt.legend()
plt.title('RMSD of Residues 40 to 56')
plt.savefig('rmsd_plot.png')
plt.show()

In [ ]:
rmsd_df.head()

### SASA

_SASA is a measure of the surface area of a protein that is accessible to the solvent (e.g., water molecules). This metric provides insights into the protein’s folding state, as buried residues typically indicate a folded structure, whereas exposed residues suggest unfolding. In this section, the SASA of the protein will be computed throughout the simulation to understand how solvent exposure changes with different conformational states._

In [ ]:
# Compute the solvent accessible surface area (SASA) for each frame
sasa = md.shrake_rupley(traj, probe_radius=0.14)  # Default probe_radius for water is 0.14 nm

# Sum SASA of all atoms to get the total SASA per frame
total_sasa_per_frame = np.sum(sasa, axis=1)

# Create DataFrame for SASA
sasa_df = pd.DataFrame(total_sasa_per_frame)

# Save the total SASA values to a CSV file
sasa_df.to_csv('sasa_values.csv', index=False)

# Plot the total SASA over time
plt.figure(figsize=(10, 6))
plt.plot(sasa_df.index, sasa_df[0], label='Total SASA', color='blue')
plt.xlabel('Frame')
plt.ylabel('Total SASA (nm^2)')
plt.title('Solvent Accessible Surface Area (SASA) Over Time')
plt.legend()
plt.grid(True)
plt.savefig('sasa_plot.png')
plt.show()

In [ ]:
sasa_df.head()

### Pairwise Distance 

_The pairwise distance between backbone atoms is a critical structural feature that can provide detailed information on the spatial arrangement of the protein's structure. In this section, we calculate the pairwise distances between selected backbone atoms (such as Cα, C, N and 0 atoms) to capture subtle changes in the protein’s conformation. These distances are particularly useful for detecting shifts in secondary structure elements (e.g., turns, loops)_

In [ ]:
# Load your trajectory and topology files
u = mda.Universe('topology.pdb', 'md_0_1_noPBC.xtc')

# Select the backbone atoms
backbone_atoms = u.select_atoms('name N C O CA')

# Initialize lists to store pairwise distances
pairwise_distances = []

# Loop over each frame in the trajectory
for ts in u.trajectory:
    # Get positions of backbone atoms
    positions = backbone_atoms.positions
    
    # Ensure all frames have the same number of backbone atoms
    n_atoms = len(positions)
    if n_atoms == 0:
        print(f"Warning: No backbone atoms found in frame {ts.frame}. Skipping this frame.")
        continue

    # Calculate pairwise distances
    distances = np.linalg.norm(positions[:, np.newaxis, :] - positions[np.newaxis, :, :], axis=-1)
    
    # Add distances to the list
    pairwise_distances.append(distances)

# Convert the list to a NumPy array for distances
pairwise_distances_arr = np.array(pairwise_distances)

# Get the number of frames and atoms
n_frames, n_atoms, _ = pairwise_distances_arr.shape

# Initialize lists for flattened distances
flattened_distances_list = []

# Flatten the pairwise distances for each frame
for distances in pairwise_distances_arr:
    flattened_distances = [distances[i, j] for i in range(n_atoms) for j in range(i + 1, n_atoms)]
    flattened_distances_list.append(flattened_distances)

# Create a DataFrame with flattened distances, using numeric indices for columns
df_distances = pd.DataFrame(flattened_distances_list)

# Set index to represent frame numbers as 0, 1, 2, ...
df_distances.index = list(range(n_frames))

# Set column names to be numeric indices 0, 1, 2, ...
df_distances.columns = list(range(df_distances.shape[1]))

# Save the combined DataFrame to a CSV file
#df_distances.to_csv('distance.csv', index=True)

In [ ]:
df_distances.head()

### Combined file

_aggregating all the extracted features—including dihedral angles, hydrogen bonds, RMSD, Rg, SASA, and pairwise distances—into a single data file. This combined dataset will serve as the input for machine learning models,enabling us to classify and analyze the conformational complexity of the GB1 protein. The data will be preprocessed, normalized, and applied PCA to facilitate efficient analysis using clustering and classification._

In [ ]:

# Combine all features into a single DataFrame
combined_df = pd.concat([angles_df, hbond_df, rg_df, rmsd_df, sasa_df, df_distances], axis=1)

# Save the combined DataFrame to a CSV file
combined_df.to_csv("combined_features.csv", index=True)
print("Combined CSV file saved as 'combined_features.csv'.")


In [ ]:
combined_df.head()

In [ ]:
combined_df.mean()

## Standardisation 

_Standardization is essential for machine learning models to ensure that all features are on a similar scale. Here Z-score standardization applied to the combined features dataset._

In [ ]:
# Standardize the combined features
features = combined_df.values
# Apply Z-score standardization
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

normalized_df= pd.DataFrame(scaled_features)

normalized_df.to_csv("standardized_combined_features.csv", index=False)
print("Standardized CSV file saved as 'standardized_combined_features.csv'.")

In [ ]:
normalized_df.head()

In [ ]:
normalized_df.mean()

In [ ]:
normalized_df.std()

# PCA

_Principal Component Analysis (PCA) is applied to the standardized data to reduce the dimensionality while retaining the maximum amount of variance from the original features. The cumulative variance explained by each principal component is computed and plotted. This plot helps determine the optimal number of components to retain, based on the amount of variance we wish to capture, aiding in the identification of key patterns in the conformational data._

In [ ]:
# Apply PCA to the scaled data
pca = PCA().fit(scaled_features)

# Compute the cumulative variance explained by each component
cumulative_variance = np.cumsum(pca.explained_variance_ratio_) * 100

# Plot the explained variance
plt.figure(figsize=(10, 6))
plt.plot(cumulative_variance)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance (%)')
plt.title('Explained Variance vs. Number of Components')
plt.grid()
plt.show()

In [ ]:
# Find the number of components that explain at least 95% of the variance

percentage_variance = 95
n_components = np.argmax(cumulative_variance >= percentage_variance) + 1

# Print the number of components
print(f'Number of components chosen to explain {percentage_variance}% variance: {n_components}')

# Apply PCA with the chosen number of components
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(scaled_features)
reconstructed_data = pca.inverse_transform(principal_components)

# Calculate reconstruction error (e.g., mean squared error)
reconstruction_error = np.mean((scaled_features- reconstructed_data) ** 2)
print(f'Reconstruction error: {reconstruction_error}')

In [ ]:
# Perform PCA and save the results
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(scaled_features)

# Create DataFrame for PCA results
pca_columns = [f'PC{i+1}' for i in range(principal_components.shape[1])]
pca_df = pd.DataFrame(data=principal_components, columns=pca_columns)

pca_df.to_csv('pca_results.csv', index=False)

# Plot all PCA components in a single figure
plt.figure(figsize=(12, 8))

# Create a pair plot of the principal components
sns.pairplot(pca_df, diag_kind='kde')

plt.suptitle('Pair Plot of PCA Components', y=1.02)
plt.tight_layout()
plt.show()

print("Explained variance ratio:", pca.explained_variance_ratio_)

In [ ]:
pca_df.describe()

# Clustering 



_In this section, K-means clustering is applied to the PCA-transformed data to identify distinct conformational clusters of the protein. The process involves the following steps:_


-  Average Inertia Calculation: A function is defined to compute the average inertia (a measure of how tightly the data points are clustered) over multiple runs of K-means for different values of k (number of clusters).

-  Elbow Method: The "elbow" point is identified by finding where the decrease in inertia begins to slow, suggesting the optimal number of clusters. The optimal value of k is determined using the elbow method, which visually inspects the plot of average inertia versus k.

-   Clustering: K-means clustering is then performed using the optimal number of clusters, and the cluster labels are assigned to the data

### 1.FInding optimal k

In [ ]:
# Define the function to calculate average inertia
def calculate_average_inertia(z, k_range, n_runs=10):
    avg_inertia = []
    for k in k_range:
        inertia_sum = 0
        for _ in range(n_runs):
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            kmeans.fit(z)
            inertia_sum += kmeans.inertia_
        avg_inertia.append(inertia_sum / n_runs)
    return avg_inertia

# Load PCA results from pca_df
z = pca_df  # Use PCA-transformed data from pca_df

# Define the range of k values
k_range = range(1, 11)

# Calculate average inertia
inertia = calculate_average_inertia(z, k_range)

# Function to find the optimal k (you can use a simpler approach without derivatives)
def find_optimal_k(inertia):
    # Find the "elbow" point where the decrease in inertia slows down
    inertia_diff = np.diff(inertia)
    inertia_diff2 = np.diff(inertia_diff)
    elbow_index = np.argmin(inertia_diff2) + 1
    return elbow_index + 1

# Determine the optimal k using the smoothed elbow method
optimal_k = find_optimal_k(inertia)
print(f'Optimal number of clusters (k) based on the elbow method: {optimal_k}')

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, color='g', marker='o', linestyle='--', linewidth=3)
plt.xlabel("Value of K")
plt.ylabel("Average Inertia")
plt.title('Elbow Method for Optimal k in K-means Clustering')
plt.xticks(k_range)
plt.grid(True)

# Annotate the optimal k value on the plot
plt.axvline(x=optimal_k, color='r', linestyle='--')
plt.text(optimal_k, inertia[k_range.index(optimal_k)], f'Optimal k = {optimal_k}', color='red', fontsize=12, ha='right')

plt.show()


### 2.k-means clustering

In [ ]:
# Apply K-means clustering using all PCA components and the optimal k
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
kmeans.fit(pca_df)  # Use all PCA components
labels = kmeans.labels_

# Add cluster labels to DataFrame
pca_df['K-means Cluster'] = labels

# Save the results with cluster labels
pca_df.to_csv('kmeans_results.csv', index=False)

### 3.visualization

In [ ]:
# Plot the clusters using the first two principal components for visualization
plt.figure(figsize=(8, 6))
sns.scatterplot(x='PC1', y='PC2', hue='K-means Cluster', palette='viridis', data=pca_df, legend='full')
plt.title(f'PCA of Protein Conformational States with K-means Clusters (k={optimal_k})')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster')
plt.show()

# Classification

_In this section, we prepare the data for classification using all the PCA components and classify the conformational states of the protein. The process involves:_

- Data Splitting: We split the dataset into training and testing sets. The feature set X includes all PCA components, while the target variable y contains the cluster labels assigned by K-means. The data is split using an 80-20 ratio, where 80% is used for training and 20% for testing.

- Defining Classifiers: We apply several classification algorithms—Support Vector Machines (SVM), K-Nearest Neighbors (KNN), Random Forest, and Logistic Regression. Each classifier is trained on the training set, and predictions are made on the test set.

- Evaluation: The performance of each classifier is evaluated using accuracy, classification reports, and confusion matrices. The accuracy measures how well the classifier predicts the correct conformational cluster. Additionally, confusion matrices are plotted to visualize the classification performance, showing the actual vs. predicted labels.

_This step helps to determine which machine learning algorithm is best suited for classifying the different conformational states of the protein based on the PCA-reduced features._

In [ ]:
# Prepare data for classification using all PCA components
X = pca_df.drop('K-means Cluster', axis=1)  # Exclude the cluster labels
y = pca_df['K-means Cluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define classifiers
classifiers = {
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression(solver='lbfgs', max_iter=1000)
}

accuracy_results = {}

for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_results[name] = accuracy
    print(f'{name} Accuracy: {accuracy}')
    print(classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{name} Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

In [ ]:
# Determine the best algorithm and rank them
best_algorithm = max(accuracy_results, key=accuracy_results.get)
sorted_algorithms = sorted(accuracy_results.items(), key=lambda item: item[1], reverse=True)

print("\nBest Algorithm:", best_algorithm)
print("Algorithm Rankings:")
for rank, (name, accuracy) in enumerate(sorted_algorithms, start=1):
    print(f'{rank}. {name}: {accuracy}')

# Visualization of the algorithm rankings
algorithms, accuracies = zip(*sorted_algorithms)
plt.figure(figsize=(10, 6))
sns.barplot(x=list(algorithms), y=list(accuracies), hue=list(algorithms), palette='viridis', legend=False)
plt.title('Algorithm Accuracy Rankings')
plt.xlabel('Algorithm')
plt.ylabel('Accuracy')

# Adding values on top of the bars
for index, value in enumerate(accuracies):
    plt.text(index, value, f'{value:.3f}', ha='center', va='bottom')

plt.show()


## cross-validation 

_In this section, we use cross-validation to further evaluate the performance and robustness of different classification algorithms. Cross-validation helps assess how well the model generalizes to unseen data by splitting the dataset into multiple subsets (folds) and testing the model on each fold._

* Cross-Validation: The cross_val_score function is used to perform 4-fold cross-validation on each classifier. This involves splitting the data into 4 parts—training the model on 3 parts and validating it on the remaining part, iterating this process 4 times so that each fold is used as a validation set once.

* Mean Accuracy: For each classifier (SVM, KNN, Random Forest, Logistic Regression), the cross-validation scores are printed along with the mean cross-validation accuracy. This gives an overall sense of how consistently the classifier performs across different subsets of the data.

_This step provides a more reliable evaluation of the classifier's performance by reducing bias and variance in the results, ensuring that the models perform well on unseen data._


In [ ]:
from sklearn.model_selection import cross_val_score , learning_curve
# Perform cross-validation and print mean accuracy
for name, clf in classifiers.items():
    scores = cross_val_score(clf, X, y, cv=4)  
    print(f'{name} Cross-Validation Scores:', scores)
    print(f'Mean {name} Cross-Validation Score:', scores.mean())
    print()

### Learning Curve

In [ ]:
from sklearn.model_selection import StratifiedKFold

# Function to plot learning curves and print scores
def plot_learning_curve_and_scores(estimator, title, X, y, cv=5, train_sizes=np.linspace(.1, 1.0, 5), random_state=42):
    plt.figure(figsize=(10, 6))
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Score")

    # Use StratifiedKFold if you have classification tasks
    stratified_cv = StratifiedKFold(n_splits=cv, shuffle=True, random_state=random_state)

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=stratified_cv, train_sizes=train_sizes,
                       return_times=True, shuffle=True, random_state=random_state)
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.grid()
    
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

    plt.legend(loc="best")
    
    # Print cross-validation scores
    scores = cross_val_score(estimator, X, y, cv=stratified_cv)
    print(f'{title} Cross-Validation Scores:', scores)
    print(f'Mean {title} Cross-Validation Score:', scores.mean())
    print()

# Example of how to use the function with classifiers
for name, clf in classifiers.items():
    plot_learning_curve_and_scores(clf, f'Learning Curve for {name}', X, y, cv=5, random_state=42)
    plt.show()


_In this section, we utilize learning curves to analyze the performance of different classifiers as the training data size increases. Learning curves provide insight into the model’s capacity, helping to understand whether a classifier is underfitting or overfitting. Additionally, we use Stratified K-Fold cross-validation to maintain the balance of class distributions across folds, which is particularly important for classification tasks with imbalanced data._

- Learning Curves: The plot_learning_curve_and_scores function is defined to plot the learning curve for each classifier. The learning curve shows how the training score and cross-validation score change as the training set size increases. 
- Stratified K-Fold Cross-Validation: The StratifiedKFold technique ensures that each fold in the cross-validation process maintains the same class distribution as the original dataset, which improves the reliability of model evaluation, especially when working with classification problems.

- Model Performance: For each classifier (SVM, KNN, Random Forest, and Logistic Regression), learning curves are plotted, and the cross-validation scores are printed. These curves help visualize how well the model generalizes with increasing amounts of training data and how its performance stabilizes across different data sizes.

# Selection and Visualization of Representative Conformations from Clusters

_In this section, we identify the best representative frame from each cluster and extract the corresponding conformation from the molecular dynamics trajectory. This allows us to visually examine the key conformational states of the protein._

- Finding Best Representative Frames:
        The centroid of each cluster, computed during K-means clustering, is considered the most representative point for that cluster.
        For each cluster, we calculate the Euclidean distance between every frame in the cluster and the centroid using the cdist function from scipy.spatial.distance.
        The frame with the smallest distance to the centroid is selected as the best representative of that cluster.

- Extracting Best Conformations:
        The best representative frames are extracted from the molecular dynamics trajectory (stored in traj), and these frames are saved to a new trajectory file (best_conformations.xtc) for further analysis.

- Visualization:
        Using NGLview, the best conformations are visualized as ribbon representations, providing an intuitive way to compare the different structural states.
        Each conformation is rendered in a 3D viewer, allowing for interactive exploration of the protein structures.

#### 1.Get the centroids of the clusters from the K-means model

In [ ]:
centroids = kmeans.cluster_centers_

#### 2.Find the closest frame to each centroid

In [ ]:
from scipy.spatial.distance import cdist
best_frames = []

for i, centroid in enumerate(centroids):
    # Get all frames in this cluster
    cluster_frames = pca_df[pca_df['K-means Cluster'] == i].drop('K-means Cluster', axis=1)
    
    # Compute the distance of each frame in the cluster to the centroid
    distances = cdist(cluster_frames, [centroid], metric='euclidean')
    
    # Find the index of the closest frame to the centroid
    best_frame_idx = cluster_frames.index[np.argmin(distances)]
    
    # Store the best frame index
    best_frames.append(best_frame_idx)

print("Best frames that represent each cluster:", best_frames)

#### 3.Extract the Best Conformations from the Trajectory

In [ ]:
# Extract the best conformations based on the frame indices
best_conformations = traj[best_frames]

# Save the best conformations to a new trajectory file 
best_conformations.save_xtc('best_conformations.xtc')


#### 3.Visualize the Best Conformations:

In [ ]:
!pip install nglview

In [ ]:
import nglview as nv
# Create a list to store views
views = []

# Create a view for each best conformation and add it to the list
for i, frame in enumerate(best_conformations):
    view = nv.show_mdtraj(frame)
    view.add_representation('ribbon')
    # view.add_representation('surface')
    views.append(view)

# Display all views
for view in views:
    display(view)

#### 4.Save each conformation as a separate PDB file

In [ ]:
for i, frame in enumerate(best_conformations):
    pdb_filename = f'best_conformation_{i}.pdb'
    frame.save(pdb_filename)
    print(f'Saved: {pdb_filename}')